# CFG

In [1]:
class CFG:
    path="../input/us-patent-phrase-to-phrase-matching"
    data='../input/v3-large-anchor-target-context-test1'
    num_workers=1
    seed=42
    batch_size=64
    lower_context=False
    dropout = 0.2
    data_type='ab-c' 
    coattention=False
    cat_gru_h=False
    cat_gru_o=False
    att_pool=True
    num_layer=1
    max_len = 214
    post=False
    
class CFG1:
    path="../input/us-patent-phrase-to-phrase-matching"
    data='../input/v1-large-anchor-target-context-test1'
    num_workers=1
    seed=42
    batch_size=64
    lower_context=False
    dropout = 0.2
    data_type='ab-c' 
    coattention=False
    cat_gru_h=False
    cat_gru_o=False
    att_pool=True
    num_layer=1
    max_len = 247
    post=False
    
class CFG2:
    path="../input/us-patent-phrase-to-phrase-matching"
    data='../input/v3-large-anchor-target'
    num_workers=1
    seed=42
    batch_size=64
    lower_context=False
    dropout = 0.2
    data_type='ab-c' 
    coattention=False
    cat_gru_h=False
    cat_gru_o=False
    att_pool=True
    num_layer=1
    max_len = 214
    post=False
    
class CFG3:
    path="../input/us-patent-phrase-to-phrase-matching"
    data='../input/bertforpatent-anchor-target'
    num_workers=1
    seed=42
    batch_size=64
    lower_context=False
    dropout = 0.2
    data_type='ab-c' 
    coattention=False
    cat_gru_h=False
    cat_gru_o=False
    att_pool=True
    num_layer=1
    max_len = 200
    post=False

# Library

In [2]:
import os
import gc
import re
import time
import math
import random
import itertools
import sys
sys.path.append("../input/debertav23fasttokenizer")
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from tqdm.auto import tqdm

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
from transformers import AutoModel, AutoConfig
from transformers import AutoTokenizer
from transformers.models.deberta.modeling_deberta import ContextPooler
from tokenization_fast import DebertaV2TokenizerFast as AutoTokenizerVF
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import gc
gc.enable()

env: TOKENIZERS_PARALLELISM=true


# Utils

In [3]:
def seed_everything(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # Two further options for CuDNN backend
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed_everything(seed=CFG.seed)

# Data Loading

In [4]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)

test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']
CFG.q_len = 44
CFG.p_len = 58 if CFG.lower_context else 85
CFG.max_len = 214
display(test.head())
tokenizer_v3 = AutoTokenizerVF.from_pretrained("../input/deberta-v3-large/deberta-v3-large")

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; alter...
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; lower loc...
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; upper porti...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# tokenizer

# Dataset

In [5]:
def sort_df_by_input_lengths(tokenizer, src_df, bs=CFG.batch_size):
    dst_df = deepcopy(src_df)

    # sort by token num
    input_lengths = []
    tk0 = tqdm(zip(dst_df['text'].fillna("").values, dst_df['context_text'].fillna("").values), total=len(dst_df))
    for text, context_text in tk0:
        length = len(tokenizer(text, context_text, add_special_tokens=True)['input_ids'])
        input_lengths.append(length)
    dst_df['input_lengths'] = input_lengths
    length_sorted_idx = np.argsort([-l for l in input_lengths])

    # sort dataframe
    sort_df = dst_df.iloc[length_sorted_idx]

    # calc max_len per batch
    sorted_input_length = sort_df['input_lengths'].values
    batch_max_length = np.zeros_like(sorted_input_length)
    for i in range((len(sorted_input_length)//bs)+1):
        batch_max_length[i*bs:(i+1)*bs] = np.max(sorted_input_length[i*bs:(i+1)*bs])    
    sort_df['batch_max_length'] = batch_max_length
    
    return sort_df, length_sorted_idx

In [6]:
def prepare_input(tokenizer, batch_max_len, text):
    inputs = tokenizer(
        text,
        add_special_tokens=True,
        max_length=batch_max_len,
        padding="max_length",
        truncation='longest_first',
        return_offsets_mapping=False
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    
    return inputs


class TestDataset(Dataset):
    def __init__(self, tokenizer, max_len, df):
        self.tokenizer = tokenizer
        self.batch_max_len = df['batch_max_length'].values
        self.texts = df['text'].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(
            self.tokenizer,
            self.batch_max_len[item],
            self.texts[item]
        )
        return inputs

# Model

In [7]:
class ClsModel(nn.Module):
    def __init__(self, model_name, config_path=None, pretrained=False):
        super().__init__()
        self.num_labels = 1
        self.cfg = CFG
        if config_path is None:
            self.config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(model_name, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)

        self.dropout = nn.Dropout(CFG.dropout)
        
        self.fc = nn.Linear(self.config.hidden_size, self.num_labels)
        self._init_weights(self.fc)

        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self.regressor = nn.Sequential(
            nn.Linear(self.config.hidden_size, 1)
        )
        self._init_weights(self.attention)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)

        sequence_output = outputs[0]      
        weights = self.attention(sequence_output)
        pooled_output = torch.sum(weights * sequence_output, dim=1)



        return pooled_output

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.regressor(feature)
        return output

In [8]:
def inference_fn(test_loader, model, device):
    model.eval()

    preds = []
    tk0 = tqdm(test_loader, total=len(test_loader))
    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
        preds.append(y_preds.sigmoid().to('cpu').numpy())

    return np.concatenate(preds)

# v3_large

In [9]:
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v3, test)
test_dataset = TestDataset(tokenizer_v3, CFG.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)



ckpts = [
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test1/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_0.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test1/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_1.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test1/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_2.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test1/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_3.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test1/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts])
for ckpt in ckpts:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_v3 = prediction[np.argsort(length_sorted_idx)]

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
##merge res by id
sort_df['deb_v3_f1']=prediction

In [11]:
xia_res1=sort_df[['id','deb_v3_f1']]

In [12]:
xia_res1.head()

,id,deb_v3_f1
18,2a619016908bfa45,0.394613
23,03ba802ed4029e4d,0.337335
35,25522ee5411e63e9,0.363062
8,faaddaf8fcba8a3f,0.384478
5,474c874d0c07bd21,0.465185


# v1_large

In [13]:
test = pd.read_csv(f'{CFG1.path}/test.csv')
submission = pd.read_csv(f'{CFG1.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG1.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)

test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']

display(test.head())
tokenizer_v1 = AutoTokenizer.from_pretrained("../input/debertalarge")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v1, test)
test_dataset = TestDataset(tokenizer_v1, CFG1.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_v1_large = [
    ("../input/debertalarge", "../input/v1-large-anchor-target-context-test1/deberta-v1-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_0.pth", 1.),
    ("../input/debertalarge", "../input/v1-large-anchor-target-context-test1/deberta-v1-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_1.pth", 1.),
    ("../input/debertalarge", "../input/v1-large-anchor-target-context-test1/deberta-v1-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_2.pth", 1.),
    ("../input/debertalarge", "../input/v1-large-anchor-target-context-test1/deberta-v1-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_3.pth", 1.),
    ("../input/debertalarge", "../input/v1-large-anchor-target-context-test1/deberta-v1-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_v1_large])
for ckpt in ckpts_v1_large:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_v1_large = prediction[np.argsort(length_sorted_idx)]
# prediction_v1_large

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; alter...
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; lower loc...
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; upper porti...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
sort_df['deb_v3_f2']=prediction
xia_res2=sort_df[['id','deb_v3_f2']]

#  v3_large_context_anchor_target

In [15]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: ";".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)

test['text'] = test['anchor'] + '[SEP]' + test['target']  +  '[SEP]' + test['context_text'] +  '[SEP]' + test['anchor_target']

display(test.head())
tokenizer_v3 = AutoTokenizerVF.from_pretrained("../input/deberta-v3-large/deberta-v3-large")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v3, test)
test_dataset = TestDataset(tokenizer_v3, CFG2.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_v3_large = [
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target/deberta-v3-large-anchor-target-310-dynamicpadding_0.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target/deberta-v3-large-anchor-target-310-dynamicpadding_1.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target/deberta-v3-large-anchor-target-310-dynamicpadding_2.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target/deberta-v3-large-anchor-target-310-dynamicpadding_3.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target/deberta-v3-large-anchor-target-310-dynamicpadding_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_v3_large])
for ckpt in ckpts_v3_large:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_v3_large_old = prediction[np.argsort(length_sorted_idx)]
# prediction_v3_large_old

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
sort_df['deb_v3_f3']=prediction
xia_res3=sort_df[['id','deb_v3_f3']]

In [17]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: ";".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)

test['text'] = test['anchor'] + '[SEP]' + test['target']  +  '[SEP]' + test['context_text'] +  '[SEP]' + test['anchor_target']

display(test.head())
tokenizer_bert = AutoTokenizer.from_pretrained("../input/bert-for-patents")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_bert, test)
test_dataset = TestDataset(tokenizer_bert, CFG3.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_bert = [
    ("../input/bert-for-patents", "../input/bertforpatent-anchor-target/bertforpatent-anchor-target-310-dynamicpadding_0.pth", 1.),
    ("../input/bert-for-patents", "../input/bertforpatent-anchor-target/bertforpatent-anchor-target-310-dynamicpadding_1.pth", 1.),
    ("../input/bert-for-patents", "../input/bertforpatent-anchor-target/bertforpatent-anchor-target-310-dynamicpadding_2.pth", 1.),
    ("../input/bert-for-patents", "../input/bertforpatent-anchor-target/bertforpatent-anchor-target-310-dynamicpadding_3.pth", 1.),
    ("../input/bert-for-patents", "../input/bertforpatent-anchor-target/bertforpatent-anchor-target-310-dynamicpadding_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_bert])
for ckpt in ckpts_bert:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_bert = prediction[np.argsort(length_sorted_idx)]
# prediction_bert

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum,PHYSICS. OPTICS,opc drum[SEP]inorganic photoconductor drum[SEP...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow[SEP]altering gas flow[SEP]MECH...
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion[SEP]lower locating[SEP]PERFORMI...
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component[SEP]upper portion[SEP]TEXTILES; ...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation[SEP]artificial neural netwo...


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
sort_df['deb_v3_f4']=prediction
xia_res4=sort_df[['id','deb_v3_f4']]

## v3 large 0619

In [19]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)


for i in range(len(test)):
    anchor_target = test['anchor_target'][i]
    target = test['target'][i]
    anchor_target_new =[element for element in anchor_target.split('; ') if element != target]
    test['anchor_target'][i] = "; ".join(anchor_target_new)


test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']

display(test.head())
tokenizer_v3 = AutoTokenizerVF.from_pretrained("../input/deberta-v3-large/deberta-v3-large")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v3, test)
test_dataset = TestDataset(tokenizer_v3, CFG2.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_v3_large = [
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_0.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_1.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_2.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_3.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_v3_large])
for ckpt in ckpts_v3_large:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_v3_large_old = prediction[np.argsort(length_sorted_idx)]
# prediction_v3_large_old



test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; [SEP...
2,36baf228038e314b,lower trunnion,lower locating,B60,,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; [SEP] PE...
3,1f37ead645e7f0c8,cap component,upper portion,D06,,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; [SEP] TEXT...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
sort_df['deb_v3_f5']=prediction
xia_res5=sort_df[['id','deb_v3_f5']]

## v1 large 0619

In [21]:
test = pd.read_csv(f'{CFG1.path}/test.csv')
submission = pd.read_csv(f'{CFG1.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG1.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)

test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']

display(test.head())
tokenizer_v1 = AutoTokenizer.from_pretrained("../input/debertalarge")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v1, test)
test_dataset = TestDataset(tokenizer_v1, CFG1.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_v1_large = [
    ("../input/debertaxlarge/deberta-xlarge", "../input/v1-xlarge-test1/deberta-v1-xlarge-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_0.pth", 1.),
    ("../input/debertaxlarge/deberta-xlarge", "../input/v1-xlarge-test1/deberta-v1-xlarge-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_1.pth", 1.),
    ("../input/debertaxlarge/deberta-xlarge", "../input/v1-xlarge-test1/deberta-v1-xlarge-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_2.pth", 1.),
    ("../input/debertaxlarge/deberta-xlarge", "../input/v1-xlarge-test1/deberta-v1-xlarge-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_3.pth", 1.),
    ("../input/debertaxlarge/deberta-xlarge", "../input/v1-xlarge-test1/deberta-v1-xlarge-anchor-target-310-dynamicpadding-anchortarget-context_text_test1_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_v1_large])
for ckpt in ckpts_v1_large:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
# prediction = np.mean(predictions, axis=0)

## data re-sort ## 
# prediction_v1_large = prediction[np.argsort(length_sorted_idx)]
# prediction_v1_large

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; alter...
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; lower loc...
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; upper porti...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
sort_df['deb_v3_f6']=prediction
xia_res6=sort_df[['id','deb_v3_f6']]

In [23]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)


for i in range(len(test)):
    anchor_target = test['anchor_target'][i]
    target = test['target'][i]
    anchor_target_new =[element for element in anchor_target.split('; ') if element != target]
    test['anchor_target'][i] = "; ".join(anchor_target_new)


test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']

display(test.head())
tokenizer_v3 = AutoTokenizerVF.from_pretrained("../input/deberta-v3-large/deberta-v3-large")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_v3, test)
test_dataset = TestDataset(tokenizer_v3, CFG2.max_len, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_v3_large = [
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2-mse/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_0.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2-mse/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_1.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2-mse/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_2.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2-mse/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_3.pth", 1.),
    ("../input/deberta-v3-large/deberta-v3-large", "../input/v3-large-anchor-target-context-test2-mse/deberta-v3-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_v3_large])
for ckpt in ckpts_v3_large:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
from sklearn.preprocessing import MinMaxScaler

MMscaler = MinMaxScaler()

prediction = MMscaler.fit_transform(prediction.reshape(-1,1)).reshape(-1)

sort_df['deb_v3_f7']=prediction
xia_res7=sort_df[['id','deb_v3_f7']]

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; [SEP...
2,36baf228038e314b,lower trunnion,lower locating,B60,,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; [SEP] PE...
3,1f37ead645e7f0c8,cap component,upper portion,D06,,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; [SEP] TEXT...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
test = pd.read_csv(f'{CFG.path}/test.csv')
submission = pd.read_csv(f'{CFG.path}/sample_submission.csv')
print(f"test.shape: {test.shape}")
print(f"submission.shape: {submission.shape}")
display(test.head())
display(submission.head())
group = test.groupby(['anchor', 'context']).agg({"target":list}).reset_index()
group["target"] = group["target"].map(lambda s: "; ".join(s))
group.rename(columns={"target":"anchor_target"}, inplace=True)
test = test.merge(group, how='left', on=["anchor", "context"])
display(test.head())
cpc_texts = torch.load(f"{CFG.data}/cpc_texts.pth")
test['context_text'] = test['context'].map(cpc_texts)


for i in range(len(test)):
    anchor_target = test['anchor_target'][i]
    target = test['target'][i]
    anchor_target_new =[element for element in anchor_target.split('; ') if element != target]
    test['anchor_target'][i] = "; ".join(anchor_target_new)


test['text'] = test['anchor'] + ' [SEP] ' + test['target']  +  '; ' + test['anchor_target'] +  ' [SEP] ' + test['context_text']

display(test.head())
tokenizer_electra = AutoTokenizer.from_pretrained("../input/electra/large-discriminator")
sort_df, length_sorted_idx = sort_df_by_input_lengths(tokenizer_electra, test)
test_dataset = TestDataset(tokenizer_electra, 225, sort_df)
test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=CFG.num_workers,
    pin_memory=True,
    drop_last=False
)

ckpts_electra = [
    ("../input/electra/large-discriminator", "../input/electra-large-anchor-target-context-test2-mse/electra-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_0.pth", 1.),
    ("../input/electra/large-discriminator", "../input/electra-large-anchor-target-context-test2-mse/electra-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_1.pth", 1.),
    ("../input/electra/large-discriminator", "../input/electra-large-anchor-target-context-test2-mse/electra-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_2.pth", 1.),
    ("../input/electra/large-discriminator", "../input/electra-large-anchor-target-context-test2-mse/electra-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_3.pth", 1.),
    ("../input/electra/large-discriminator", "../input/electra-large-anchor-target-context-test2-mse/electra-large-anchor-target-310-dynamicpadding-anchortarget-context_text_test2_mse_4.pth", 1.),
]

predictions = []
weight_sum = sum([ckpt[2] for ckpt in ckpts_electra])
for ckpt in ckpts_electra:
    model = ClsModel(ckpt[0], config_path=None, pretrained=False).to(device)
    model.load_state_dict(torch.load(ckpt[1], map_location=device)['model'])
    prediction = inference_fn(test_loader, model, device)
    if ckpt[2] != 1:
        prediction *= ckpt[2]
    predictions.append(prediction)
    del model, prediction
    gc.collect()
    torch.cuda.empty_cache()
prediction = np.sum(predictions, axis=0) / weight_sum
from sklearn.preprocessing import MinMaxScaler

MMscaler = MinMaxScaler()

prediction = MMscaler.fit_transform(prediction.reshape(-1,1)).reshape(-1)

sort_df['deb_v3_f8']=prediction
xia_res8=sort_df[['id','deb_v3_f8']]

test.shape: (36, 4)
submission.shape: (36, 2)


,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


,id,score
0,4112d61851461f60,0
1,09e418c93a776564,0
2,36baf228038e314b,0
3,1f37ead645e7f0c8,0
4,71a5b6ad068d531f,0


,id,anchor,target,context,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,inorganic photoconductor drum
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,altering gas flow
2,36baf228038e314b,lower trunnion,lower locating,B60,lower locating
3,1f37ead645e7f0c8,cap component,upper portion,D06,upper portion
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,artificial neural network


,id,anchor,target,context,anchor_target,context_text,text
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,,PHYSICS. OPTICS,opc drum [SEP] inorganic photoconductor drum; ...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,adjust gas flow [SEP] altering gas flow; [SEP...
2,36baf228038e314b,lower trunnion,lower locating,B60,,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,lower trunnion [SEP] lower locating; [SEP] PE...
3,1f37ead645e7f0c8,cap component,upper portion,D06,,TEXTILES; PAPER. TREATMENT OF TEXTILES OR THE ...,cap component [SEP] upper portion; [SEP] TEXT...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,,ELECTRICITY. ELECTRIC COMMUNICATION TECHNIQUE,neural stimulation [SEP] artificial neural net...


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

# Saber


## Saber DEBERTA-V3-LARGE CV8475 LB8524

In [25]:
import re
import os
import time
import random

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import torch
from transformers import AutoConfig, AutoTokenizer, AutoModel
class Setting:
    pretrained_model_path = "../input/deberta-v3-large/deberta-v3-large"
    output_model_path = "../input/pppm-xjbt-pcc-shuffle"
    train_folds = [0, 1, 2, 3, 4]

    max_len = 180
    valid_batch_size = 128
    
cfg = Setting()
random.seed(2022)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {device}")

def anchor_target_random_shuffle(target, anchor_target):
    shuffled = [element for element in anchor_target if element != target]
    shuffled = random.sample(shuffled, min(len(shuffled), 29))
    shuffled = shuffled + [target]
    random.shuffle(shuffled)
    
    return shuffled

class PatentPhraseDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.is_train = is_train
        self.tokenizer = tokenizer

        self.anchor = df["anchor"].tolist()
        self.target = df["target"].tolist()
        self.context = df["context_texts"].tolist()
        self.anchor_target = df["anchor_target"].tolist()

        if self.is_train:
            self.score = df["score"].tolist()

    def __getitem__(self, index):
        anchor = self.anchor[index]
        target = self.target[index]
        context = self.context[index]
        
        anchor_target = self.anchor_target[index]
        anchor_target = ";".join(anchor_target_random_shuffle(target, anchor_target))

        inputs = anchor + "[SEP]" + target + ". " + anchor_target + "[SEP]" + context

        encoded = self.tokenizer(
            inputs,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_token_type_ids=False,
        )

        if self.is_train:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
                "labels": torch.FloatTensor([self.score[index]])
            }
        else:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
            }
        
    
    def __len__(self):
        return len(self.anchor)
class PatentPhraseModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(cfg.pretrained_model_path)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })

        self.bert = AutoModel.from_pretrained(cfg.pretrained_model_path, config=config)
        self.regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(config.hidden_size, 1)
        )

    def forward(self, ids, masks):
        bert_output = self.bert(input_ids=ids, attention_mask=masks)
        last_hidden_states = bert_output.last_hidden_state
        
        return self.regressor(last_hidden_states[:, 0])
    
dtype = {
    "id": "str",
    "anchor": "str",
    "target": "str",
    "score": "float32",
}

titles = pd.read_csv("../input/cpc-codes/titles.csv")
first_class_mapping = titles.loc[titles["code"].isin(["A", "B", "C", "D", "E", "F", "G", "H", "Y"]), ["code", "title"]]
first_class_mapping = dict(zip(first_class_mapping["code"], first_class_mapping["title"]))

test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv", dtype=dtype)
test = test.merge(titles, left_on="context", right_on="code")

test["section_name"] = test["section"].map(first_class_mapping)
test["context_texts"] = test["section_name"] + ". " + test["title"]

group = test.groupby(["anchor", "context"]).agg({"target": list}).reset_index()
group.rename(columns={"target": "anchor_target"}, inplace=True)

test = test.merge(group, how="left", on=["anchor", "context"])

test.head()



Device is cuda


,id,anchor,target,context,code,title,section,class,subclass,group,main_group,section_name,context_texts,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS,[inorganic photoconductor drum]
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS,[generate by layer]
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS,[illumination]
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,[altering gas flow]
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...,[lower locating]


In [26]:
def data_fn(data, tokenizer):
    test_dataset = PatentPhraseDataset(data, tokenizer, is_train=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, shuffle=False, drop_last=False, batch_size=cfg.valid_batch_size,
    )

    return test_loader


def test_fn(test_loader, model, device):
    test_array = list()

    model.eval()
    for inputs in test_loader:
        ids, masks = inputs["ids"].to(device), inputs["masks"].to(device)
        with torch.no_grad():
            output = model(ids=ids, masks=masks)

        test_array.append(output.sigmoid().cpu().numpy().ravel())

    return np.concatenate(test_array)

In [27]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_model_path)
test_loader = data_fn(test, tokenizer)

net = PatentPhraseModel()
net.to(device)

predictions = list()
scale_predictions = list()

for idx in cfg.train_folds:
    start = time.perf_counter()
    model_path = os.path.join(cfg.output_model_path, f"{idx + 1}.pth")
    net.load_state_dict(torch.load(model_path, map_location=device))

    output = test_fn(test_loader, net, device)
    duration = time.perf_counter() - start
    print(f"fold={idx + 1}/{len(cfg.train_folds)}, model='{model_path}', time={duration:.2f}s")
    print(output[:6])
    predictions.append(output.copy())
    
    output = MinMaxScaler().fit_transform(output.reshape(-1, 1)).ravel()
    print(output[:6])
    scale_predictions.append(output.copy())
# v3_saber = np.mean(scale_predictions, 0)
# v3_saber

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.classifer.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifer.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exa

fold=1/5, model='../input/pppm-xjbt-pcc-shuffle/1.pth', time=14.52s
[0.5866403  0.61911523 0.55943567 0.6132171  0.5814282  0.6067595 ]
[0.5640297  0.7183473  0.43475604 0.69032    0.5392623  0.6596341 ]
fold=2/5, model='../input/pppm-xjbt-pcc-shuffle/2.pth', time=14.94s
[0.5022665  0.5906488  0.49350688 0.54357463 0.48427558 0.55119306]
[0.5270257  0.83845675 0.49615943 0.6725824  0.46363127 0.69942725]
fold=3/5, model='../input/pppm-xjbt-pcc-shuffle/3.pth', time=14.82s
[0.5534198  0.62118936 0.52954906 0.5966998  0.5497288  0.56626064]
[0.54150593 0.8165897  0.4446119  0.7171837  0.5265235  0.59362805]
fold=4/5, model='../input/pppm-xjbt-pcc-shuffle/4.pth', time=15.58s
[0.4997991  0.578462   0.47946224 0.5492282  0.49687573 0.57367855]
[0.49268222 0.77966344 0.41848838 0.6730114  0.48201704 0.76221216]
fold=5/5, model='../input/pppm-xjbt-pcc-shuffle/5.pth', time=13.78s
[0.5289517  0.5995934  0.49183086 0.58936757 0.5483584  0.57829076]
[0.5082518  0.7485105  0.38200045 0.71373165 0.5

In [28]:
oof_train = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv", dtype={"id": "str", "label": "float32"})
oof_test = test[["id"]].copy()

In [29]:
feat_name = "saber_f1"
tmp = pd.read_csv(f"{cfg.output_model_path}/oof_df.csv").rename(columns={"oof_score": feat_name})
oof_train = oof_train.merge(tmp[["id", feat_name]], how="left", on="id")
oof_train

,id,anchor,target,context,score,saber_f1
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,0.500093
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,0.566894
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,0.242008
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,0.525668
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,0.076151
...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,0.940370
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,0.204819
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,0.413685
36471,756ec035e694722b,wood article,wooden material,B44,0.75,0.715330


In [30]:
test[feat_name] = np.mean(scale_predictions, 0)
oof_test = oof_test.merge(test[["id", feat_name]], how="left", on="id")
oof_test.head(10)

,id,saber_f1
0,4112d61851461f60,0.526699
1,5203a36c501f1b7c,0.780313
2,7aa5908a77a7ec24,0.435203
3,09e418c93a776564,0.693366
4,36baf228038e314b,0.517138
5,b892011ab2e2cabc,0.678192
6,1f37ead645e7f0c8,0.392335
7,71a5b6ad068d531f,0.287170
8,16ae4b99d3601e60,0.268165
9,474c874d0c07bd21,0.496197


## Saber ELECTRA-LARGE CV8408 LB?

In [31]:
class Setting:
    pretrained_model_path = "../input/electra/large-discriminator"
    output_model_path = "../input/pppm-xjbt-electra-pcc"
    train_folds = [0, 1, 2, 3, 4]

    max_len = 200
    valid_batch_size = 128
    
cfg = Setting()
random.seed(791736)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {device}")

Device is cuda


In [32]:
def anchor_target_random_shuffle(target, anchor_target):
    shuffled = [element for element in anchor_target if element != target]
    shuffled = random.sample(shuffled, min(len(shuffled), 29))
    shuffled = shuffled + [target]
    random.shuffle(shuffled)
    
    return shuffled


class PatentPhraseDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.is_train = is_train
        self.tokenizer = tokenizer

        self.anchor = df["anchor"].tolist()
        self.target = df["target"].tolist()
        self.context = df["context_texts"].tolist()
        self.anchor_target = df["anchor_target"].tolist()

        if self.is_train:
            self.score = df["score"].tolist()

    def __getitem__(self, index):
        anchor = self.anchor[index]
        target = self.target[index]
        context = self.context[index]
        
        anchor_target = self.anchor_target[index]
        anchor_target = ";".join(anchor_target_random_shuffle(target, anchor_target))

        inputs = anchor + "[SEP]" + target + "; " + anchor_target + "[SEP]" + context

        encoded = self.tokenizer(
            inputs,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_token_type_ids=False,
        )

        if self.is_train:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
                "labels": torch.FloatTensor([self.score[index]])
            }
        else:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
            }
        
    
    def __len__(self):
        return len(self.anchor)

In [33]:
class PatentPhraseModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(cfg.pretrained_model_path)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })

        self.bert = AutoModel.from_pretrained(cfg.pretrained_model_path, config=config)
        self.regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(config.hidden_size, 1)
        )

    def forward(self, ids, masks):
        bert_output = self.bert(input_ids=ids, attention_mask=masks)
        last_hidden_states = bert_output.last_hidden_state
        
        return self.regressor(last_hidden_states[:, 0])

In [34]:
dtype = {
    "id": "str",
    "anchor": "str",
    "target": "str",
    "score": "float32",
}

titles = pd.read_csv("../input/cpc-codes/titles.csv")
first_class_mapping = titles.loc[titles["code"].isin(["A", "B", "C", "D", "E", "F", "G", "H", "Y"]), ["code", "title"]]
first_class_mapping = dict(zip(first_class_mapping["code"], first_class_mapping["title"]))

test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv", dtype=dtype)
test = test.merge(titles, left_on="context", right_on="code")

test["section_name"] = test["section"].map(first_class_mapping)
test["context_texts"] = test["section_name"] + "; " + test["title"]

group = test.groupby(["anchor", "context"]).agg({"target": list}).reset_index()
group.rename(columns={"target": "anchor_target"}, inplace=True)

test = test.merge(group, how="left", on=["anchor", "context"])

test.head()

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,section_name,context_texts,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[inorganic photoconductor drum]
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[generate by layer]
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[illumination]
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,[altering gas flow]
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING,PERFORMING OPERATIONS; TRANSPORTING; VEHICLES ...,[lower locating]


In [35]:
def data_fn(data, tokenizer):
    test_dataset = PatentPhraseDataset(data, tokenizer, is_train=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, shuffle=False, drop_last=False, batch_size=cfg.valid_batch_size,
    )

    return test_loader


def test_fn(test_loader, model, device):
    test_array = list()

    model.eval()
    for inputs in test_loader:
        ids, masks = inputs["ids"].to(device), inputs["masks"].to(device)
        with torch.no_grad():
            output = model(ids=ids, masks=masks)

        test_array.append(output.cpu().numpy().ravel())

    return np.concatenate(test_array)

In [36]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_model_path)
test_loader = data_fn(test, tokenizer)

net = PatentPhraseModel()
net.to(device)

predictions = list()
scale_predictions = list()

for idx in cfg.train_folds:
    start = time.perf_counter()
    model_path = os.path.join(cfg.output_model_path, f"{idx + 1}.pth")
    net.load_state_dict(torch.load(model_path, map_location=device))

    output = test_fn(test_loader, net, device)
    duration = time.perf_counter() - start
    print(f"fold={idx + 1}/{len(cfg.train_folds)}, model='{model_path}', time={duration:.2f}s")
    print(output[:6])
    predictions.append(output.copy())
    
    output = MinMaxScaler().fit_transform(output.reshape(-1, 1)).ravel()
    print(output[:6])
    scale_predictions.append(output.copy())

Some weights of the model checkpoint at ../input/electra/large-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


fold=1/5, model='../input/pppm-xjbt-electra-pcc/1.pth', time=10.77s
[-0.01668248  0.14954266 -0.02164822  0.14389893  0.00322013  0.16106462]
[0.421207   0.7286767  0.41202182 0.7182374  0.45802122 0.74998903]
fold=2/5, model='../input/pppm-xjbt-electra-pcc/2.pth', time=10.29s
[ 0.09369694  0.16012481 -0.02830411  0.09115988  0.00766607  0.09671962]
[0.62964976 0.75441515 0.40050632 0.6248846  0.46806583 0.635327  ]
fold=3/5, model='../input/pppm-xjbt-electra-pcc/3.pth', time=10.27s
[ 0.02443628  0.13033494 -0.02702917  0.12060457 -0.01888605  0.09874868]
[0.49684033 0.7296108  0.38371676 0.7082229  0.40161574 0.6601826 ]
fold=4/5, model='../input/pppm-xjbt-electra-pcc/4.pth', time=10.59s
[ 0.01276425  0.13114055 -0.05397929  0.11960667 -0.0266099   0.11467342]
[0.49271864 0.7386948  0.3540311  0.71472836 0.4109024  0.7044775 ]
fold=5/5, model='../input/pppm-xjbt-electra-pcc/5.pth', time=10.47s
[ 0.0430315   0.10591101  0.0079382   0.11173703 -0.01386292  0.11670449]
[0.5362731  0.6775

In [37]:
feat_name = "saber_f2"
tmp = pd.read_csv(f"{cfg.output_model_path}/oof_df.csv").rename(columns={"oof_score": feat_name})
oof_train = oof_train.merge(tmp[["id", feat_name]], how="left", on="id")
oof_train

,id,anchor,target,context,score,saber_f1,saber_f2
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,0.500093,0.571102
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,0.566894,0.609552
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,0.242008,0.213746
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,0.525668,0.530854
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,0.076151,0.069262
...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,0.940370,0.983885
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,0.204819,0.465405
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,0.413685,0.420911
36471,756ec035e694722b,wood article,wooden material,B44,0.75,0.715330,0.645753


In [38]:
test[feat_name] = np.mean(scale_predictions, 0)
oof_test = oof_test.merge(test[["id", feat_name]], how="left", on="id")
oof_test.head(10)

,id,saber_f1,saber_f2
0,4112d61851461f60,0.526699,0.515338
1,5203a36c501f1b7c,0.780313,0.725794
2,7aa5908a77a7ec24,0.435203,0.401538
3,09e418c93a776564,0.693366,0.691347
4,36baf228038e314b,0.517138,0.429406
5,b892011ab2e2cabc,0.678192,0.690360
6,1f37ead645e7f0c8,0.392335,0.299719
7,71a5b6ad068d531f,0.287170,0.336468
8,16ae4b99d3601e60,0.268165,0.285546
9,474c874d0c07bd21,0.496197,0.535167


## Saber DERBATA-V3-LARGE CV8505 LB?

In [39]:
class Setting:
    pretrained_model_path = "../input/deberta-v3-large/deberta-v3-large"
    output_model_path = "../input/pppm-xjbt-pcc-cat-rm"
    train_folds = [0, 1, 2, 3, 4]

    max_len = 200
    valid_batch_size = 128
    
cfg = Setting()
random.seed(819361)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {device}")

Device is cuda


In [40]:
def anchor_target_random_shuffle(target, anchor_target):
    shuffled = [element for element in anchor_target if element != target]
    shuffled = random.sample(shuffled, min(len(shuffled), 29))
    # shuffled = shuffled + [target]
    random.shuffle(shuffled)
    
    return shuffled


class PatentPhraseDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.is_train = is_train
        self.tokenizer = tokenizer

        self.anchor = df["anchor"].tolist()
        self.target = df["target"].tolist()
        self.context = df["context_texts"].tolist()
        self.anchor_target = df["anchor_target"].tolist()

        if self.is_train:
            self.score = df["score"].tolist()

    def __getitem__(self, index):
        anchor = self.anchor[index]
        target = self.target[index]
        context = self.context[index]
        
        anchor_target = self.anchor_target[index]
        anchor_target = ";".join(anchor_target_random_shuffle(target, anchor_target))

        inputs = anchor + "[SEP]" + target + "; " + anchor_target + "[SEP]" + context

        encoded = self.tokenizer(
            inputs,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_token_type_ids=False,
        )

        if self.is_train:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
                "labels": torch.FloatTensor([self.score[index]])
            }
        else:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
            }
        
    
    def __len__(self):
        return len(self.anchor)

In [41]:
class PatentPhraseModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(cfg.pretrained_model_path)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })

        self.bert = AutoModel.from_pretrained(cfg.pretrained_model_path, config=config)
        self.regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(config.hidden_size, 1)
        )

    def forward(self, ids, masks):
        bert_output = self.bert(input_ids=ids, attention_mask=masks)
        last_hidden_states = bert_output.last_hidden_state
        
        return self.regressor(last_hidden_states[:, 0])

In [42]:
dtype = {
    "id": "str",
    "anchor": "str",
    "target": "str",
    "score": "float32",
}

titles = pd.read_csv("../input/cpc-codes/titles.csv")
first_class_mapping = titles.loc[titles["code"].isin(["A", "B", "C", "D", "E", "F", "G", "H", "Y"]), ["code", "title"]]
first_class_mapping = dict(zip(first_class_mapping["code"], first_class_mapping["title"]))

test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv", dtype=dtype)
test = test.merge(titles, left_on="context", right_on="code")

test["section_name"] = test["section"].map(first_class_mapping)
test["context_texts"] = test["section_name"] + "; " + test["title"]

group = test.groupby(["anchor", "context"]).agg({"target": list}).reset_index()
group.rename(columns={"target": "anchor_target"}, inplace=True)

test = test.merge(group, how="left", on=["anchor", "context"])

test.head()

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,section_name,context_texts,anchor_target
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[inorganic photoconductor drum]
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[generate by layer]
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS; OPTICS,[illumination]
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,[altering gas flow]
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING,PERFORMING OPERATIONS; TRANSPORTING; VEHICLES ...,[lower locating]


In [43]:
def data_fn(data, tokenizer):
    test_dataset = PatentPhraseDataset(data, tokenizer, is_train=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, shuffle=False, drop_last=False, batch_size=cfg.valid_batch_size,
    )

    return test_loader


def test_fn(test_loader, model, device):
    test_array = list()

    model.eval()
    for inputs in test_loader:
        ids, masks = inputs["ids"].to(device), inputs["masks"].to(device)
        with torch.no_grad():
            output = model(ids=ids, masks=masks)

        test_array.append(output.cpu().numpy().ravel())

    return np.concatenate(test_array)

In [44]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_model_path)
test_loader = data_fn(test, tokenizer)

net = PatentPhraseModel()
net.to(device)

predictions = list()
scale_predictions = list()

for idx in cfg.train_folds:
    start = time.perf_counter()
    model_path = os.path.join(cfg.output_model_path, f"{idx + 1}.pth")
    net.load_state_dict(torch.load(model_path, map_location=device))

    output = test_fn(test_loader, net, device)
    duration = time.perf_counter() - start
    print(f"fold={idx + 1}/{len(cfg.train_folds)}, model='{model_path}', time={duration:.2f}s")
    print(output[:6])
    predictions.append(output.copy())
    
    output = MinMaxScaler().fit_transform(output.reshape(-1, 1)).ravel()
    print(output[:6])
    scale_predictions.append(output.copy())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.classifer.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifer.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exa

fold=1/5, model='../input/pppm-xjbt-pcc-cat-rm/1.pth', time=13.95s
[0.2688807  0.61680216 0.01902756 0.42478147 0.16170532 0.42706025]
[0.57561725 0.8659736  0.3671034  0.7057235  0.4861745  0.70762527]
fold=2/5, model='../input/pppm-xjbt-pcc-cat-rm/2.pth', time=14.14s
[ 0.0492351   0.30221352 -0.18054223  0.10485224 -0.00309585  0.04507936]
[0.62446195 0.8719546  0.39966726 0.67887306 0.5732658  0.6203963 ]
fold=3/5, model='../input/pppm-xjbt-pcc-cat-rm/3.pth', time=13.74s
[ 0.28975818  0.51179755 -0.01133345  0.42557526  0.19210003  0.3785409 ]
[0.61638725 0.8020303  0.36464998 0.7299415  0.53473705 0.6906169 ]
fold=4/5, model='../input/pppm-xjbt-pcc-cat-rm/4.pth', time=14.60s
[0.24899541 0.5594518  0.12843457 0.44946623 0.31826395 0.5027432 ]
[0.52380294 0.79623586 0.4180079  0.69972086 0.5845877  0.7464727 ]
fold=5/5, model='../input/pppm-xjbt-pcc-cat-rm/5.pth', time=14.01s
[0.46691272 0.5118776  0.27445304 0.52591753 0.4498902  0.5242503 ]
[0.63455486 0.6884085  0.4040496  0.70522

In [45]:
feat_name = "saber_f3"
tmp = pd.read_csv(f"{cfg.output_model_path}/oof_df.csv").rename(columns={"oof_score": feat_name})
oof_train = oof_train.merge(tmp[["id", feat_name]], how="left", on="id")
oof_train

,id,anchor,target,context,score,saber_f1,saber_f2,saber_f3
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,0.500093,0.571102,0.525286
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,0.566894,0.609552,0.578426
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,0.242008,0.213746,0.292639
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,0.525668,0.530854,0.438175
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,0.076151,0.069262,0.083204
...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,0.940370,0.983885,0.966474
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,0.204819,0.465405,0.293934
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,0.413685,0.420911,0.412662
36471,756ec035e694722b,wood article,wooden material,B44,0.75,0.715330,0.645753,0.623055


In [46]:
test[feat_name] = np.mean(scale_predictions, 0)
oof_test = oof_test.merge(test[["id", feat_name]], how="left", on="id")
oof_test.head(10)

,id,saber_f1,saber_f2,saber_f3
0,4112d61851461f60,0.526699,0.515338,0.594965
1,5203a36c501f1b7c,0.780313,0.725794,0.804921
2,7aa5908a77a7ec24,0.435203,0.401538,0.390696
3,09e418c93a776564,0.693366,0.691347,0.703897
4,36baf228038e314b,0.517138,0.429406,0.558586
5,b892011ab2e2cabc,0.678192,0.690360,0.693668
6,1f37ead645e7f0c8,0.392335,0.299719,0.376955
7,71a5b6ad068d531f,0.287170,0.336468,0.266759
8,16ae4b99d3601e60,0.268165,0.285546,0.307841
9,474c874d0c07bd21,0.496197,0.535167,0.474013


## Saber DERBATA-V3-LARGE CV8311 LB?

In [47]:
class Setting:
    pretrained_model_path = "../input/deberta-v3-large/deberta-v3-large"
    output_model_path = "../input/pppm-final-pcc"
    train_folds = [0, 1, 2, 3, 4]

    max_len = 64
    valid_batch_size = 128
    
cfg = Setting()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device is {device}")

Device is cuda


In [48]:
class PatentPhraseDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.is_train = is_train
        self.tokenizer = tokenizer

        self.anchor = df["anchor"].tolist()
        self.target = df["target"].tolist()
        self.context = df["context_texts"].tolist()

        if self.is_train:
            self.score = df["score"].tolist()

    def __getitem__(self, index):
        anchor = self.anchor[index]
        target = self.target[index]
        context = self.context[index]

        inputs = anchor + "[SEP]" + target + "[SEP]" + context

        encoded = self.tokenizer(
            inputs,
            max_length=cfg.max_len,
            padding="max_length",
            truncation=True,
            return_token_type_ids=False,
        )

        if self.is_train:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
                "labels": torch.FloatTensor([self.score[index]])
            }
        else:
            return {
                "ids": torch.LongTensor(encoded["input_ids"]),
                "masks": torch.LongTensor(encoded["attention_mask"]),
            }
        
    
    def __len__(self):
        return len(self.anchor)

In [49]:
class PatentPhraseModel(torch.nn.Module):
    def __init__(self):
        super().__init__()

        config = AutoConfig.from_pretrained(cfg.pretrained_model_path)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })

        self.bert = AutoModel.from_pretrained(cfg.pretrained_model_path, config=config)
        self.regressor = torch.nn.Sequential(
            torch.nn.Dropout(0.1),
            torch.nn.Linear(config.hidden_size, 1)
        )

    def forward(self, ids, masks):
        bert_output = self.bert(input_ids=ids, attention_mask=masks)
        last_hidden_states = bert_output.last_hidden_state
        
        return self.regressor(last_hidden_states[:, 0])

In [50]:
dtype = {
    "id": "str",
    "anchor": "str",
    "target": "str",
    "score": "float32",
}

titles = pd.read_csv("../input/cpc-codes/titles.csv")
first_class_mapping = titles.loc[titles["code"].isin(["A", "B", "C", "D", "E", "F", "G", "H", "Y"]), ["code", "title"]]
first_class_mapping = dict(zip(first_class_mapping["code"], first_class_mapping["title"]))

test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv", dtype=dtype)
test = test.merge(titles, left_on="context", right_on="code")

test["section_name"] = test["section"].map(first_class_mapping)
test["context_texts"] = test["section_name"] + ". " + test["title"]

test.head()

,id,anchor,target,context,code,title,section,class,subclass,group,main_group,section_name,context_texts
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS
1,5203a36c501f1b7c,generate in layer,generate by layer,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS
2,7aa5908a77a7ec24,el display,illumination,G02,G02,OPTICS,G,2.0,NaN,NaN,NaN,PHYSICS,PHYSICS. OPTICS
3,09e418c93a776564,adjust gas flow,altering gas flow,F23,F23,COMBUSTION APPARATUS; COMBUSTION PROCESSES,F,23.0,NaN,NaN,NaN,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...,MECHANICAL ENGINEERING; LIGHTING; HEATING; WEA...
4,36baf228038e314b,lower trunnion,lower locating,B60,B60,VEHICLES IN GENERAL,B,60.0,NaN,NaN,NaN,PERFORMING OPERATIONS; TRANSPORTING,PERFORMING OPERATIONS; TRANSPORTING. VEHICLES ...


In [51]:
def data_fn(data, tokenizer):
    test_dataset = PatentPhraseDataset(data, tokenizer, is_train=False)

    test_loader = torch.utils.data.DataLoader(
        test_dataset, shuffle=False, drop_last=False, batch_size=cfg.valid_batch_size,
    )

    return test_loader


def test_fn(test_loader, model, device):
    test_array = list()

    model.eval()
    for inputs in test_loader:
        ids, masks = inputs["ids"].to(device), inputs["masks"].to(device)
        with torch.no_grad():
            output = model(ids=ids, masks=masks)

        test_array.append(output.cpu().numpy().ravel())

    return np.concatenate(test_array)

In [52]:
tokenizer = AutoTokenizer.from_pretrained(cfg.pretrained_model_path)
test_loader = data_fn(test, tokenizer)

net = PatentPhraseModel()
net.to(device)

predictions = list()
scale_predictions = list()

for idx in cfg.train_folds:
    start = time.perf_counter()
    model_path = os.path.join(cfg.output_model_path, f"{idx + 1}.pth")
    net.load_state_dict(torch.load(model_path, map_location=device))

    output = test_fn(test_loader, net, device)
    duration = time.perf_counter() - start
    print(f"fold={idx + 1}/{len(cfg.train_folds)}, model='{model_path}', time={duration:.2f}s")
    print(output[:6])
    predictions.append(output.copy())
    
    output = MinMaxScaler().fit_transform(output.reshape(-1, 1)).ravel()
    print(output[:6])
    scale_predictions.append(output.copy())

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at ../input/deberta-v3-large/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.dense.weight', 'mask_predictions.classifer.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.classifer.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exa

fold=1/5, model='../input/pppm-final-pcc/1.pth', time=12.92s
[ 0.16427939  0.37404343 -0.04312945  0.22532946  0.01862881  0.16398716]
[0.60442793 0.89143956 0.3206388  0.68796027 0.40514013 0.6040281 ]
fold=2/5, model='../input/pppm-final-pcc/2.pth', time=13.29s
[ 0.12643349  0.2661765  -0.06854392  0.25698695  0.02022175  0.09113456]
[0.6016294  0.7644675  0.3744284  0.7537592  0.47786424 0.5604967 ]
fold=3/5, model='../input/pppm-final-pcc/3.pth', time=13.92s
[ 0.03332779  0.3068186  -0.10143549  0.188238    0.0209475   0.15446703]
[0.50941384 0.8729811  0.33026516 0.71534497 0.49295598 0.6704513 ]
fold=4/5, model='../input/pppm-final-pcc/4.pth', time=14.01s
[ 0.04073207  0.35802022 -0.0091744   0.3261069   0.11086499  0.2862382 ]
[0.38587335 0.7986558  0.3209465  0.75713754 0.4771142  0.7052695 ]
fold=5/5, model='../input/pppm-final-pcc/5.pth', time=13.84s
[ 0.09663812  0.1965869  -0.14857627  0.19222954  0.00484675  0.1195583 ]
[0.6326335  0.7578658  0.32538852 0.7524062  0.517622

In [53]:
feat_name = "saber_f4"
tmp = pd.read_csv(f"{cfg.output_model_path}/oof_df.csv").rename(columns={"oof_score": feat_name})
oof_train = oof_train.merge(tmp[["id", feat_name]], how="left", on="id")
oof_train

,id,anchor,target,context,score,saber_f1,saber_f2,saber_f3,saber_f4
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,0.500093,0.571102,0.525286,0.398648
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,0.566894,0.609552,0.578426,0.675260
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,0.242008,0.213746,0.292639,0.267790
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,0.525668,0.530854,0.438175,0.468767
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,0.076151,0.069262,0.083204,0.099338
...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,0.940370,0.983885,0.966474,0.949454
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,0.204819,0.465405,0.293934,0.482302
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,0.413685,0.420911,0.412662,0.486823
36471,756ec035e694722b,wood article,wooden material,B44,0.75,0.715330,0.645753,0.623055,0.648915


In [54]:
test[feat_name] = np.mean(scale_predictions, 0)
oof_test = oof_test.merge(test[["id", feat_name]], how="left", on="id")
oof_test.head(10)

,id,saber_f1,saber_f2,saber_f3,saber_f4
0,4112d61851461f60,0.526699,0.515338,0.594965,0.546796
1,5203a36c501f1b7c,0.780313,0.725794,0.804921,0.817082
2,7aa5908a77a7ec24,0.435203,0.401538,0.390696,0.334333
3,09e418c93a776564,0.693366,0.691347,0.703897,0.733322
4,36baf228038e314b,0.517138,0.429406,0.558586,0.474139
5,b892011ab2e2cabc,0.678192,0.690360,0.693668,0.640319
6,1f37ead645e7f0c8,0.392335,0.299719,0.376955,0.322511
7,71a5b6ad068d531f,0.287170,0.336468,0.266759,0.092022
8,16ae4b99d3601e60,0.268165,0.285546,0.307841,0.248047
9,474c874d0c07bd21,0.496197,0.535167,0.474013,0.513344


In [55]:
## xia res

In [56]:
xia_res=pd.merge(xia_res1,xia_res2,on='id',how='inner')

In [57]:
xia_res=pd.merge(xia_res,xia_res3,on='id',how='inner')

In [58]:
xia_res=pd.merge(xia_res,xia_res4,on='id',how='inner')

In [59]:
xia_res=pd.merge(xia_res,xia_res5,on='id',how='inner')

In [60]:
xia_res=pd.merge(xia_res,xia_res6,on='id',how='inner')

In [61]:
xia_res=pd.merge(xia_res,xia_res7,on='id',how='inner')

In [62]:
xia_res=pd.merge(xia_res,xia_res8,on='id',how='inner')

In [63]:
final_res=pd.merge(xia_res,oof_test,on='id',how='inner')

In [64]:
final_res

,id,deb_v3_f1,deb_v3_f2,deb_v3_f3,deb_v3_f4,deb_v3_f5,deb_v3_f6,deb_v3_f7,deb_v3_f8,saber_f1,saber_f2,saber_f3,saber_f4
0,2a619016908bfa45,0.394613,0.377453,0.418248,0.376272,0.315584,0.386982,0.368700,0.344734,0.409600,0.395741,0.367788,0.474989
1,03ba802ed4029e4d,0.337335,0.263719,0.247699,0.225164,0.267782,0.238087,0.229486,0.203291,0.236642,0.260646,0.291563,0.232045
2,25522ee5411e63e9,0.363062,0.316239,0.275951,0.296037,0.275003,0.287985,0.292632,0.238227,0.250655,0.294922,0.307029,0.222879
3,faaddaf8fcba8a3f,0.384478,0.250740,0.338662,0.375462,0.331274,0.283149,0.443260,0.330814,0.318418,0.395242,0.338956,0.305756
4,474c874d0c07bd21,0.465185,0.496792,0.459988,0.521699,0.379865,0.427696,0.539571,0.528834,0.496197,0.535167,0.474013,0.513344
5,c404f8b378cbb008,0.527898,0.521590,0.500655,0.533441,0.434684,0.475446,0.572861,0.479753,0.511938,0.519218,0.512743,0.516178
6,ae0262c02566d2ce,0.910626,0.992105,0.933813,0.995406,0.866505,0.982784,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,26c3c6dc6174b589,0.006640,0.029056,0.002989,0.078520,0.077770,0.173000,0.000000,0.111743,0.012707,0.126176,0.010550,0.016443
8,09e418c93a776564,0.627622,0.707367,0.662315,0.646644,0.569003,0.667481,0.737586,0.675445,0.693366,0.691347,0.703897,0.733322
9,1f37ead645e7f0c8,0.391267,0.326637,0.335390,0.324254,0.329421,0.354401,0.410915,0.283154,0.392335,0.299719,0.376955,0.322511


In [65]:
# deb_v3_f1 	deb_v3_f2 	deb_v3_f3 	deb_v3_f4 	saber_deberta
# deb_v3_f1 	deb_v3_f2 	deb_v3_f3 	deb_v3_f4 	deb_v3_f5 	deb_v3_f6 	saber_f1 	saber_f2 	saber_f3 	saber_f4
# deb_v3_f1 	deb_v3_f2 	deb_v3_f3 	deb_v3_f4 	deb_v3_f5 	deb_v3_f6 	deb_v3_f7 	deb_v3_f8 	saber_f1 	saber_f2 	saber_f3 	saber_f4
final_res['score']=(final_res['deb_v3_f1']+final_res['deb_v3_f2']+final_res['deb_v3_f3']+final_res['deb_v3_f4']+final_res['deb_v3_f5']+final_res['deb_v3_f6']+final_res['deb_v3_f7']+final_res['deb_v3_f8']+final_res['saber_f1']+final_res['saber_f2']+final_res['saber_f3']+final_res['saber_f4'])/12.0

In [66]:
final_res

,id,deb_v3_f1,deb_v3_f2,deb_v3_f3,deb_v3_f4,deb_v3_f5,deb_v3_f6,deb_v3_f7,deb_v3_f8,saber_f1,saber_f2,saber_f3,saber_f4,score
0,2a619016908bfa45,0.394613,0.377453,0.418248,0.376272,0.315584,0.386982,0.368700,0.344734,0.409600,0.395741,0.367788,0.474989,0.385892
1,03ba802ed4029e4d,0.337335,0.263719,0.247699,0.225164,0.267782,0.238087,0.229486,0.203291,0.236642,0.260646,0.291563,0.232045,0.252788
2,25522ee5411e63e9,0.363062,0.316239,0.275951,0.296037,0.275003,0.287985,0.292632,0.238227,0.250655,0.294922,0.307029,0.222879,0.285052
3,faaddaf8fcba8a3f,0.384478,0.250740,0.338662,0.375462,0.331274,0.283149,0.443260,0.330814,0.318418,0.395242,0.338956,0.305756,0.341351
4,474c874d0c07bd21,0.465185,0.496792,0.459988,0.521699,0.379865,0.427696,0.539571,0.528834,0.496197,0.535167,0.474013,0.513344,0.486529
5,c404f8b378cbb008,0.527898,0.521590,0.500655,0.533441,0.434684,0.475446,0.572861,0.479753,0.511938,0.519218,0.512743,0.516178,0.508867
6,ae0262c02566d2ce,0.910626,0.992105,0.933813,0.995406,0.866505,0.982784,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.973437
7,26c3c6dc6174b589,0.006640,0.029056,0.002989,0.078520,0.077770,0.173000,0.000000,0.111743,0.012707,0.126176,0.010550,0.016443,0.053800
8,09e418c93a776564,0.627622,0.707367,0.662315,0.646644,0.569003,0.667481,0.737586,0.675445,0.693366,0.691347,0.703897,0.733322,0.676283
9,1f37ead645e7f0c8,0.391267,0.326637,0.335390,0.324254,0.329421,0.354401,0.410915,0.283154,0.392335,0.299719,0.376955,0.322511,0.345580


# Submission

In [67]:
# final_res[['id', 'score']].to_csv('submission.csv', index=False)

In [68]:
# all_predictions = [
# #     (prediction_v1, 1.),
# #     (prediction_v2, 1.),
#     (prediction_v3, 1.)
# ]
# weight_sum = sum([c[1] for c in all_predictions])
# prediction = np.sum([c[0]*c[1] for c in all_predictions], axis=0) / weight_sum

# submission['score'] = predictions

# submission[['id', 'score']].to_csv('submission.csv', index=False)

## Mean

In [69]:
xia_deberta_v3=pd.read_csv("../input/v3-large-anchor-target-context-test1/oof_df.csv")
oof_xia_deberta_v3=xia_deberta_v3[['id','pred','score']]

In [70]:
xia_deberta_2=pd.read_csv("../input/v1-large-anchor-target-context-test1/oof_df.csv")
oof_xia_deberta_2=xia_deberta_2[['id','pred']]

In [71]:
xia_deberta_3=pd.read_csv("../input/v3-large-anchor-target/oof_df.csv")
oof_xia_deberta_3=xia_deberta_3[['id','pred']]

In [72]:
xia_deberta_4=pd.read_csv("../input/bertforpatent-anchor-target/oof_df.csv")
oof_xia_deberta_4=xia_deberta_4[['id','pred']]

In [73]:
xia_deberta_5=pd.read_csv("../input/v3-large-anchor-target-context-test2/oof_df.csv")
oof_xia_deberta_5=xia_deberta_5[['id','pred']]

In [74]:
xia_deberta_6=pd.read_csv("../input/v1-xlarge-test1/oof_df.csv")
oof_xia_deberta_6=xia_deberta_6[['id','pred']]

In [75]:
xia_deberta_7=pd.read_csv("../input/v3-large-anchor-target-context-test2-mse/oof_df.csv")
oof_xia_deberta_7=xia_deberta_7[['id','pred']]
oof_xia_deberta_7['pred'] = MinMaxScaler().fit_transform(oof_xia_deberta_7['pred'].values.reshape(-1, 1)).ravel()

In [76]:
xia_deberta_8=pd.read_csv("../input/electra-large-anchor-target-context-test2-mse/oof_df.csv")
oof_xia_deberta_8=xia_deberta_8[['id','pred']]
oof_xia_deberta_8['pred'] = MinMaxScaler().fit_transform(oof_xia_deberta_8['pred'].values.reshape(-1, 1)).ravel()

In [77]:
merge_train_pred=pd.merge(oof_xia_deberta_v3,oof_xia_deberta_2,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_3,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_4,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_5,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_6,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_7,how='inner',on='id')
merge_train_pred=pd.merge(merge_train_pred,oof_xia_deberta_8,how='inner',on='id')

In [78]:
merge_train_pred

,id,pred_x,score,pred_y,pred_x,pred_y,pred_x,pred_y,pred_x,pred_y
0,37d61fd2272659b1,0.581905,0.50,0.517203,0.467623,0.685268,0.613386,0.661623,0.592101,0.617486
1,7b9652b17b68b7a4,0.636381,0.75,0.631971,0.632748,0.569967,0.652195,0.707445,0.700893,0.693657
2,36d72442aefd8232,0.310578,0.25,0.234594,0.236493,0.259771,0.259242,0.226501,0.286177,0.272720
3,5296b0c19e1ce60e,0.498577,0.50,0.559883,0.404845,0.586742,0.448168,0.606233,0.533984,0.526212
4,54c1e3b9184cb5b6,0.005480,0.00,0.011305,0.030650,0.002237,0.074732,0.006287,0.173077,0.209928
...,...,...,...,...,...,...,...,...,...,...
36468,685a74e244440afa,0.531098,0.75,0.642656,0.460201,0.593353,0.559469,0.513372,0.574831,0.628785
36469,cb06d60871f0b271,0.559316,0.75,0.600599,0.366687,0.589098,0.533920,0.419402,0.602833,0.312608
36470,aaacebd906186f88,0.577954,0.75,0.586943,0.363362,0.591866,0.536970,0.437407,0.611738,0.311733
36471,523630d2783fd4c4,0.208217,0.00,0.188955,0.045982,0.245977,0.072020,0.184296,0.131282,0.062109


In [79]:
merge_train_pred.columns=['id','f1','score','f2','f3','f4','f5',"f6","f7","f8"]

In [80]:
oof_train

,id,anchor,target,context,score,saber_f1,saber_f2,saber_f3,saber_f4
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,0.500093,0.571102,0.525286,0.398648
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,0.566894,0.609552,0.578426,0.675260
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,0.242008,0.213746,0.292639,0.267790
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,0.525668,0.530854,0.438175,0.468767
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,0.076151,0.069262,0.083204,0.099338
...,...,...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,0.940370,0.983885,0.966474,0.949454
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,0.204819,0.465405,0.293934,0.482302
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,0.413685,0.420911,0.412662,0.486823
36471,756ec035e694722b,wood article,wooden material,B44,0.75,0.715330,0.645753,0.623055,0.648915


In [81]:
oof_train.columns

Index(['id', 'anchor', 'target', 'context', 'score', 'saber_f1', 'saber_f2', 'saber_f3', 'saber_f4'], dtype='object')

In [82]:
oof_train_pd=oof_train[['id','saber_f1', 'saber_f2','saber_f3', 'saber_f4']]

In [83]:
merge_train_pred=pd.merge(merge_train_pred,oof_train_pd,how='inner',on='id')

In [84]:
merge_train_pred

,id,f1,score,f2,f3,f4,f5,f6,f7,f8,saber_f1,saber_f2,saber_f3,saber_f4
0,37d61fd2272659b1,0.581905,0.50,0.517203,0.467623,0.685268,0.613386,0.661623,0.592101,0.617486,0.500093,0.571102,0.525286,0.398648
1,7b9652b17b68b7a4,0.636381,0.75,0.631971,0.632748,0.569967,0.652195,0.707445,0.700893,0.693657,0.566894,0.609552,0.578426,0.675260
2,36d72442aefd8232,0.310578,0.25,0.234594,0.236493,0.259771,0.259242,0.226501,0.286177,0.272720,0.242008,0.213746,0.292639,0.267790
3,5296b0c19e1ce60e,0.498577,0.50,0.559883,0.404845,0.586742,0.448168,0.606233,0.533984,0.526212,0.525668,0.530854,0.438175,0.468767
4,54c1e3b9184cb5b6,0.005480,0.00,0.011305,0.030650,0.002237,0.074732,0.006287,0.173077,0.209928,0.076151,0.069262,0.083204,0.099338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36468,685a74e244440afa,0.531098,0.75,0.642656,0.460201,0.593353,0.559469,0.513372,0.574831,0.628785,0.531517,0.535169,0.538188,0.557738
36469,cb06d60871f0b271,0.559316,0.75,0.600599,0.366687,0.589098,0.533920,0.419402,0.602833,0.312608,0.412603,0.512333,0.444951,0.522559
36470,aaacebd906186f88,0.577954,0.75,0.586943,0.363362,0.591866,0.536970,0.437407,0.611738,0.311733,0.447757,0.536501,0.486376,0.575862
36471,523630d2783fd4c4,0.208217,0.00,0.188955,0.045982,0.245977,0.072020,0.184296,0.131282,0.062109,0.095451,0.079716,0.041737,0.167986


In [85]:
merge_train_pred.columns=['id','f1','score','f2','f3','f4','f5',"f6","f7","f8","f9","f10","f11","f12"]

In [86]:
merge_train_pred.head()

,id,f1,score,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,37d61fd2272659b1,0.581905,0.50,0.517203,0.467623,0.685268,0.613386,0.661623,0.592101,0.617486,0.500093,0.571102,0.525286,0.398648
1,7b9652b17b68b7a4,0.636381,0.75,0.631971,0.632748,0.569967,0.652195,0.707445,0.700893,0.693657,0.566894,0.609552,0.578426,0.675260
2,36d72442aefd8232,0.310578,0.25,0.234594,0.236493,0.259771,0.259242,0.226501,0.286177,0.272720,0.242008,0.213746,0.292639,0.267790
3,5296b0c19e1ce60e,0.498577,0.50,0.559883,0.404845,0.586742,0.448168,0.606233,0.533984,0.526212,0.525668,0.530854,0.438175,0.468767
4,54c1e3b9184cb5b6,0.005480,0.00,0.011305,0.030650,0.002237,0.074732,0.006287,0.173077,0.209928,0.076151,0.069262,0.083204,0.099338


In [87]:
merge_train_pred['score_avg']=(merge_train_pred['f1']+merge_train_pred['f2']+merge_train_pred['f3']+merge_train_pred['f4']+merge_train_pred['f5']+merge_train_pred['f6']+merge_train_pred['f7']+merge_train_pred['f8']+merge_train_pred['f9']+merge_train_pred['f10']+merge_train_pred['f11']+merge_train_pred['f12'])/12.0

In [88]:
import scipy as sp
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score


In [89]:
get_score(merge_train_pred['score'],merge_train_pred['score_avg'])

0.8673949344990008

## stacking

In [90]:
x_train = pd.DataFrame(
    np.vstack((merge_train_pred['f1'].values, merge_train_pred['f2'].values, merge_train_pred['f3'].values, merge_train_pred['f4'].values, merge_train_pred['f5'].values,merge_train_pred['f6'].values, merge_train_pred['f7'].values, merge_train_pred['f8'].values, merge_train_pred['f9'].values, merge_train_pred['f10'].values, merge_train_pred['f11'].values, merge_train_pred['f12'].values)).T,
    columns=["f1", "f2", "f3","f4","f5","f6", "f7", "f8","f9","f10","f11",'f12']
)

In [91]:
x_train

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,0.581905,0.517203,0.467623,0.685268,0.613386,0.661623,0.592101,0.617486,0.500093,0.571102,0.525286,0.398648
1,0.636381,0.631971,0.632748,0.569967,0.652195,0.707445,0.700893,0.693657,0.566894,0.609552,0.578426,0.675260
2,0.310578,0.234594,0.236493,0.259771,0.259242,0.226501,0.286177,0.272720,0.242008,0.213746,0.292639,0.267790
3,0.498577,0.559883,0.404845,0.586742,0.448168,0.606233,0.533984,0.526212,0.525668,0.530854,0.438175,0.468767
4,0.005480,0.011305,0.030650,0.002237,0.074732,0.006287,0.173077,0.209928,0.076151,0.069262,0.083204,0.099338
...,...,...,...,...,...,...,...,...,...,...,...,...
36468,0.531098,0.642656,0.460201,0.593353,0.559469,0.513372,0.574831,0.628785,0.531517,0.535169,0.538188,0.557738
36469,0.559316,0.600599,0.366687,0.589098,0.533920,0.419402,0.602833,0.312608,0.412603,0.512333,0.444951,0.522559
36470,0.577954,0.586943,0.363362,0.591866,0.536970,0.437407,0.611738,0.311733,0.447757,0.536501,0.486376,0.575862
36471,0.208217,0.188955,0.045982,0.245977,0.072020,0.184296,0.131282,0.062109,0.095451,0.079716,0.041737,0.167986


In [92]:
y_train = merge_train_pred["score"].values
y_train

array([0.5 , 0.75, 0.25, ..., 0.75, 0.  , 0.75])

In [93]:
# deb_v3_f1 	deb_v3_f2 	deb_v3_f3 	deb_v3_f4 	deb_v3_f5 	deb_v3_f6 	saber_f1 	saber_f2 	saber_f3 	saber_f4
x_test = pd.DataFrame(
    np.vstack((final_res['deb_v3_f1'].values,final_res['deb_v3_f2'].values,final_res['deb_v3_f3'].values,final_res['deb_v3_f4'].values,final_res['deb_v3_f5'].values,final_res['deb_v3_f6'].values,final_res['deb_v3_f7'].values,final_res['deb_v3_f8'].values,final_res['saber_f1'].values,final_res['saber_f2'].values,final_res['saber_f3'].values,final_res['saber_f4'].values)).T,
    columns=["f1", "f2", "f3","f4","f5","f6", "f7", "f8","f9","f10","f11","f12"]
)

In [94]:
x_test

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12
0,0.394613,0.377453,0.418248,0.376272,0.315584,0.386982,0.368700,0.344734,0.409600,0.395741,0.367788,0.474989
1,0.337335,0.263719,0.247699,0.225164,0.267782,0.238087,0.229486,0.203291,0.236642,0.260646,0.291563,0.232045
2,0.363062,0.316239,0.275951,0.296037,0.275003,0.287985,0.292632,0.238227,0.250655,0.294922,0.307029,0.222879
3,0.384478,0.250740,0.338662,0.375462,0.331274,0.283149,0.443260,0.330814,0.318418,0.395242,0.338956,0.305756
4,0.465185,0.496792,0.459988,0.521699,0.379865,0.427696,0.539571,0.528834,0.496197,0.535167,0.474013,0.513344
5,0.527898,0.521590,0.500655,0.533441,0.434684,0.475446,0.572861,0.479753,0.511938,0.519218,0.512743,0.516178
6,0.910626,0.992105,0.933813,0.995406,0.866505,0.982784,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
7,0.006640,0.029056,0.002989,0.078520,0.077770,0.173000,0.000000,0.111743,0.012707,0.126176,0.010550,0.016443
8,0.627622,0.707367,0.662315,0.646644,0.569003,0.667481,0.737586,0.675445,0.693366,0.691347,0.703897,0.733322
9,0.391267,0.326637,0.335390,0.324254,0.329421,0.354401,0.410915,0.283154,0.392335,0.299719,0.376955,0.322511


In [95]:
from sklearn.model_selection import KFold, StratifiedKFold
feats = x_train.columns.tolist()
n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=12)

In [96]:
stack_train = np.zeros(x_train.shape[0])
stack_test = np.zeros(x_test.shape[0])
feats_impt = np.zeros(len(x_train.columns.tolist()))

In [97]:
from sklearn.linear_model import LinearRegression, Ridge
import pickle
for i, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train*4)):
    model = LinearRegression()
    model.fit(x_train.iloc[train_idx], y_train[train_idx])
    test_score = model.predict(x_test)

    stack_train[valid_idx] = model.predict(x_train.iloc[valid_idx])
    stack_test += test_score
    feats_impt += model.coef_

#     with open(f"stack_model{i + 1}.pkl", "wb") as f:
#         pickle.dump(model, f)

In [98]:
# import lightgbm as lgb
# import pickle

# def lgb_pcc(y_true, y_pred):
#     return 'pcc', sp.stats.pearsonr(y_pred, y_true)[0], True

# for i, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train*4)):
#     print(f'Stacking: {i + 1}/{5}... ')
#     model = lgb.LGBMRegressor(
#         boosting_type='gbdt',
#         importance_type='gain',
#         learning_rate=0.01,
#         max_depth=5,
#         num_leaves=20,
#         bagging_freq=5,
#         bagging_fraction=0.6,
#         feature_fraction=0.4,
#         random_state=110,
#         bagging_seed=119,
#         n_estimators=500,
#     )
#     model.fit(
#         x_train.iloc[train_idx], y_train[train_idx],
#         eval_set=[(x_train.iloc[valid_idx], y_train[valid_idx])],
#         eval_metric=lambda y_true, y_pred: [lgb_pcc(y_true, y_pred)],
#         verbose=100,
#     )
#     test_score = model.predict(x_test)
    
#     stack_train[valid_idx] = model.predict(x_train.iloc[valid_idx])
#     stack_test += test_score
#     feats_impt += model.feature_importances_

#     with open(f"stack_model{i + 1}.pkl", "wb") as f:
#         pickle.dump(model, f)

In [99]:
stack_test /= n_splits
feats_impt /= n_splits

In [100]:
importance = pd.DataFrame(zip(feats, feats_impt), columns=["feat", "score"]) \
    .sort_values("score", ascending=False) \
    .reset_index(drop=True)
print(importance.head(20).to_markdown())

|    | feat   |     score |
|---:|:-------|----------:|
|  0 | f8     | 0.22804   |
|  1 | f7     | 0.156927  |
|  2 | f11    | 0.133572  |
|  3 | f4     | 0.129032  |
|  4 | f6     | 0.0902782 |
|  5 | f12    | 0.0692138 |
|  6 | f9     | 0.0623287 |
|  7 | f1     | 0.0564957 |
|  8 | f3     | 0.0433054 |
|  9 | f5     | 0.0238503 |
| 10 | f10    | 0.0185592 |
| 11 | f2     | 0.0110956 |


In [101]:
import scipy as sp
def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score
get_score(stack_train, y_train)

0.8683275477839625

In [102]:
stack_test

array([ 0.3680569 ,  0.22770833,  0.26560669,  0.34079682,  0.49277381,
        0.50557268,  0.99046349,  0.04069443,  0.68095559,  0.33478703,
        0.19537829,  0.71503436, -0.00597141,  0.29539043,  0.06284229,
        0.71981523,  0.40950904,  0.03977815,  0.67430042,  0.17079993,
        0.28041459,  0.48555383,  0.33149725,  0.34483248,  0.63989783,
        0.32454239,  0.27718385,  0.45362846,  0.28958491,  0.56334592,
        0.41538877,  0.77255916,  0.0601477 ,  0.34311525,  0.47986121,
        0.40546556])

In [103]:
final_res['stack_score']=stack_test

In [104]:
final_res_stack=final_res[['id', 'stack_score']]
final_res_stack.columns=['id','score']

In [105]:
final_res_stack

,id,score
0,2a619016908bfa45,0.368057
1,03ba802ed4029e4d,0.227708
2,25522ee5411e63e9,0.265607
3,faaddaf8fcba8a3f,0.340797
4,474c874d0c07bd21,0.492774
5,c404f8b378cbb008,0.505573
6,ae0262c02566d2ce,0.990463
7,26c3c6dc6174b589,0.040694
8,09e418c93a776564,0.680956
9,1f37ead645e7f0c8,0.334787


In [106]:
final_res_stack[['id', 'score']].to_csv('submission.csv', index=False)

In [107]:
# merge_train_pred.to_csv('oof_11.csv', index=False)